In [ ]:
import random
random.seed(42)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt

import torch
from torch import optim, nn
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
from torchvision.utils import make_grid
from torchvision import transforms as T
from torchvision import models, datasets
from random import randint

from tqdm import tqdm
import os

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

In [ ]:
DATA_DIR = 'tiny-imagenet-200' # Original images come in shapes of [3,64,64]

# Define training and validation data paths
TRAIN_DIR = os.path.join(DATA_DIR, 'train') 
TEST_DIR = os.path.join(DATA_DIR, 'val')

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
def show_batch(dataloader):
    dataiter = iter(dataloader)
    images, labels = dataiter.next()    
    imshow(make_grid(images)) # Using Torchvision.utils make_grid function
    
def show_image(dataloader):
    dataiter = iter(dataloader)
    images, labels = dataiter.next()
    random_num = randint(0, len(images)-1)
    imshow(images[random_num])
    label = labels[random_num]
    print(f'Label: {label}, Shape: {images[random_num].shape}')
    

In [ ]:
# Setup function to create dataloaders for image datasets
def generate_dataloader(data, name, transform, batch_size=32, val_split=None):
    if data is None: 
        return None
    
    # Read image files to pytorch dataset using ImageFolder, a generic data 
    # loader where images are in format root/label/filename
    # See https://pytorch.org/vision/stable/datasets.html
    if transform is None:
        dataset = datasets.ImageFolder(data, transform=T.ToTensor())
    else:
        dataset = datasets.ImageFolder(data, transform=transform)

    if val_split:
        val_ds_length = int(len(dataset)*val_split)
        train_ds_length = len(dataset) - val_ds_length
        train_split_dataset, val_split_dataset = random_split(dataset, [train_ds_length, val_ds_length], generator=torch.Generator().manual_seed(42))
        print(f'Number of training samples: {len(train_split_dataset)}, Number of unique train labels: {len(set(train_split_dataset.dataset.targets))}')
        print(f'Number of validation samples: {len(val_split_dataset)}, Number of unique validation labels: {len(set(val_split_dataset.dataset.targets))}')

    # Set options for device
    if use_cuda:
        kwargs = {"pin_memory": True, "num_workers": 1}
    else:
        kwargs = {}
    
    if not val_split:
      # Wrap image dataset (defined above) in dataloader 
      dataloader = DataLoader(dataset, batch_size=batch_size, 
                          shuffle=(name=="train"), 
                          **kwargs)
      
      return dataloader
    else:
        train_dataloader = DataLoader(train_split_dataset, batch_size=batch_size, 
                          shuffle=(name=="train"), 
                          **kwargs)
        val_dataloader = DataLoader(val_split_dataset, batch_size=batch_size, 
                          shuffle=(name=="train"), 
                          **kwargs)
        return train_dataloader, val_dataloader

In [ ]:
# Create separate validation subfolders for the validation images based on
# their labels indicated in the val_annotations txt file
test_img_dir = os.path.join(TEST_DIR, 'images')

# Open and read val annotations text file
fp = open(os.path.join(TEST_DIR, 'val_annotations.txt'), 'r')
data = fp.readlines()

# Create dictionary to store img filename (word 0) and corresponding
# label (word 1) for every line in the txt file (as key value pair)
val_img_dict = {}
for line in data:
    words = line.split('\t')
    val_img_dict[words[0]] = words[1]
fp.close()

# Display first 10 entries of resulting val_img_dict dictionary
print({k: val_img_dict[k] for k in list(val_img_dict)[:10]})

# Create subfolders (if not present) for validation images based on label,
# and move images into the respective folders
for img, folder in val_img_dict.items():
    newpath = (os.path.join(test_img_dir, folder))
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    if os.path.exists(os.path.join(test_img_dir, img)):
        os.rename(os.path.join(test_img_dir, img), os.path.join(newpath, img))

In [ ]:
mean = [0.4802369434455423, 0.4480670369274663, 0.39750364210899203]
std = [0.2764364260576532, 0.268863276261085, 0.28158992530131444]

In [ ]:
# Transformation - optional depending on future resnet implementation

# Define transformation sequence for image pre-processing
# If not using pre-trained model, normalize with 0.5, 0.5, 0.5 (mean and SD)
# If using pre-trained ImageNet, normalize with mean=[0.485, 0.456, 0.406], 
# std=[0.229, 0.224, 0.225])

train_transform = T.Compose([
    # T.Resize(32), # Resize images to 256 x 256
    # T.CenterCrop(224), # Center crop image
    T.RandomHorizontalFlip(),
    T.RandomRotation(15),
    T.ToTensor(),
    T.Normalize(mean, std)
])

test_transform = T.Compose([
                # T.Resize(32), # Resize images to 256 x 256
                # T.CenterCrop(224), # Center crop image
                # T.RandomHorizontalFlip(),
                T.ToTensor(),  # Converting cropped images to tensors
                T.Normalize(mean, std)
])


In [ ]:
batch_size = 128

In [ ]:

from PIL.Image import NONE
#
# Define batch size for DataLoaders
batch_size = 128

# Create DataLoaders for pre-trained models (normalized based on specific requirements)
train_loader = generate_dataloader(TRAIN_DIR, "train",
                                  transform=train_transform, batch_size=batch_size)

test_loader = generate_dataloader(test_img_dir, "val",
                                 transform=test_transform, batch_size=batch_size)


In [ ]:
show_image(train_loader)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

    
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Input size: [batch, 3, 32, 32]
        # Output size: [batch, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),           # [batch, 24, 8, 8]
            nn.ReLU(),
            nn.Conv2d(24, 48, 4, stride=2, padding=1),           # [batch, 48, 4, 4]
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
            nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded.view(-1, 48*8*8)

def test():
    net = Autoencoder()
    net = net.to(device)
    item = iter(train_loader)
    img, _ = next(item)

    img = img[0]
    img = img[None, :]
    img = img.to(device)
    img_cons, rep  = net(img)
    std_t = torch.tensor(std).to(device)
    mean_t = torch.tensor(mean).to(device)
    img_cons = img_cons * std_t[:, None, None] + mean_t[:, None, None]
    img = img * std_t[:, None, None] + mean_t[:, None, None]
    # print(rep)
    print(rep.shape, "  ", img_cons.shape)
    temp_r = imshow(img.cpu()[0])
    imshow(img_cons.detach().cpu()[0])

In [ ]:
test()

In [ ]:
dataloaders = {
    'train': train_loader,
    'val': test_loader,
    'test': test_loader
}
dataset_sizes = {
    'train': len(train_loader.dataset),
    'val': len(test_loader.dataset),
    'test': len(test_loader.dataset),
}

In [ ]:
import time
import copy
from torch import autograd

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train','val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            for inputs, _ in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    reconstructed_img, hidden_rep = model(inputs)
                    
                    # l1_loss = sparse_loss(model, inputs) * 0.001
                    loss = criterion(reconstructed_img, inputs)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        with autograd.detect_anomaly():
                            loss.backward()
                            optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss}')          

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
#             if phase == 'val':
                print('Saving..')
                state = {
                    'model': model.state_dict(),
                    'loss': epoch_loss,
                    'epoch': epoch,
                }
                # if not os.path.isdir('checkpoint'):
                #     os.mkdir('checkpoint')
                torch.save(state, './ae_tiny_imagenet.pth')
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val loss: {best_loss:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
from torch.optim import lr_scheduler
model_ft = Autoencoder()
model_ft = model_ft.to(device)

criterion = nn.MSELoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
exp_lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_ft, milestones=[60, 120, 160], gamma=0.2)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=200)

In [ ]:
import cv2
device = 'cuda'
model_ft.eval()
item = iter(train_loader)
img, _ = next(item)

img = img[0]
img = img[None, :]
img = img.to(device)
img_cons, rep  = model_ft(img)
std_t = torch.tensor(std).to(device)
mean_t = torch.tensor(mean).to(device)
img_cons = img_cons * std_t[:, None, None] + mean_t[:, None, None]
img = img * std_t[:, None, None] + mean_t[:, None, None]
print(rep.shape, "  ", img_cons.shape)
temp_r = imshow(img.cpu()[0])
imshow(img_cons.detach().cpu()[0])
print(rep)

In [ ]:
train_image_embeddings = []
train_image_labels = []

def populate_embedding_lable_list(model):
    global train_image_embeddings, train_image_labels

    model.eval()   # Set model to evaluate mode
    # Iterate over data.
    for inputs, labels in tqdm(dataloaders['train']):
        inputs = inputs.to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            reconstructed_img, hidden_rep = model(inputs)
            train_image_embeddings.extend(hidden_rep)
            train_image_labels.extend(labels)

In [ ]:
populate_embedding_lable_list(model_ft)

In [ ]:
train_image_labels = torch.flatten(torch.tensor(train_image_labels))

train_image_embeddings_tensor = torch.stack(train_image_embeddings)

In [ ]:
print(len(sorted(set(list(train_loader.dataset.targets)))))
print(len(sorted(set(list(train_image_labels.numpy())))))
print(train_image_labels.shape, " ", train_image_embeddings_tensor.shape)

In [ ]:
from collections import Counter

def find_indices(list_to_check, item_to_find):
    return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]
  
image_label_list = list(train_image_labels.numpy())
classes_to_index = {i: [] for i in range(200)}
for key in classes_to_index.keys():
    indexes = find_indices(image_label_list, key)
    random_40 = random.sample(indexes,40)
    classes_to_index[key] = random_40

print(classes_to_index)

In [ ]:
similarity_matrix = np.zeros((200,200,2))

In [ ]:
cosine_sim = nn.CosineSimilarity(dim=0, eps=1e-8)
print(train_image_embeddings[0].shape)
cosine_sim(train_image_embeddings[2957], train_image_embeddings[20364].T)

In [ ]:
import itertools
final_indices = list(classes_to_index.values())
final_indices = sorted([item for sublist in final_indices for item in sublist])
print(final_indices)
print(len(final_indices))

In [ ]:
for act_i in tqdm(range(len(final_indices))):
    for act_j in range(act_i+1, len(final_indices)):
        i, j = final_indices[act_i], final_indices[act_j]
        class_i, class_j = train_image_labels[i], train_image_labels[j]
        emb_i, emb_j = train_image_embeddings[i], train_image_embeddings[j]
        sim_score = cosine_sim(emb_i, emb_j).detach().item()
    
        if class_i != class_j:
            similarity_matrix[class_i][class_j][0] += sim_score
            similarity_matrix[class_i][class_j][1] += 1

            similarity_matrix[class_j][class_i][0] += sim_score
            similarity_matrix[class_j][class_i][1] += 1
        else:
            similarity_matrix[class_i][class_i][0] += sim_score
            similarity_matrix[class_i][class_i][1] += 1

In [ ]:
avg_sim = np.zeros((200,200))
for i in range(200):
    for j in range(200):
        if i == j:
            avg_sim[i][j] = similarity_matrix[i][j][0]/similarity_matrix[i][j][1] + 0.0225
        else:
            avg_sim[i][j] = similarity_matrix[i][j][0]/similarity_matrix[i][j][1]

In [ ]:
import torch.nn.functional as F
x_distribution = F.softmax(torch.tensor(avg_sim), dim = 1)

In [ ]:
count = 0
for i in range(len(x_distribution)):
    if i == torch.argmax(x_distribution[i]):
        count += 1
        # print(i)
print(count)

In [ ]:
torch.save(avg_sim, './tiny_imagenet_ae_simmatrix.pt')